In [1]:
from bs4 import BeautifulSoup

In [2]:
import re

In [3]:
html = '''

<html>
 <head></head>
 <body>
  <ul>
   <LI><a href=link1.html>내용1</p></li>
   <li><a href=link2.html>내용2</p></Li>
   <li><a href=link3.html>내용3</p>
   <li><a href=link4.html>내용4</p></LI>
   </li>
  </ul>
 </body>
</html>

'''

In [4]:
BeautifulSoup(html, 'html.parser')


<html>
<head></head>
<body>
<ul>
<li><a href="link1.html">내용1</a></li></ul></body></html>
<li><a href="link2.html">내용2</a></li>
<li><a href="link3.html">내용3</a></li>
<li><a href="link4.html">내용4</a></li>





In [5]:
BeautifulSoup(html, 'lxml')

<html>
<head></head>
<body>
<ul>
<li><a href="link1.html">내용1</a></li>
<li><a href="link2.html">내용2</a></li>
<li><a href="link3.html">내용3
   <li><a href="link4.html">내용4</a></li>
</a></li>
</ul>
</body>
</html>

In [6]:
BeautifulSoup(html, 'html5lib')

<html><head></head>
 <body>
  <ul>
   <li><a href="link1.html">내용1<p></p></a></li><a href="link1.html">
   </a><li><a href="link1.html"></a><a href="link2.html">내용2<p></p></a></li><a href="link2.html">
   </a><li><a href="link2.html"></a><a href="link3.html">내용3<p></p>
   </a></li><li><a href="link4.html">내용4<p></p></a></li><a href="link4.html">
   
  </a></ul><a href="link4.html">
 


</a></body></html>

In [7]:
dom = BeautifulSoup(html, 'html.parser')

In [8]:
type(dom)

bs4.BeautifulSoup

In [9]:
dom.html.body.ul.li.p
#html 의 관계를 순차적으로 내림차순!

In [10]:
dom.p
#아니면 그냥 이렇게 첫번째 매칭한 애를 리턴하게 할수도 있다! 그리디

In [11]:
dom.a == dom.find_all('a')[1]

False

In [12]:
dom.find('a')

<a href="link1.html">내용1</a>

In [13]:
dom.find(attrs={'href':'link1.html'})

<a href="link1.html">내용1</a>

In [14]:
dom.find('a', recursive = False)
#못찾는다! 이게 왜필요할까?
#태그가 중복될때 자식은 됬고 자신만 불러올때 필요하다~

In [15]:
dom.find(text = '내용1').find_parent()

<a href="link1.html">내용1</a>

In [16]:
dom.find(text = '내용1').find_parent('a')

<a href="link1.html">내용1</a>

In [17]:
dom.find_all('a',limit=2)

[<a href="link1.html">내용1</a>, <a href="link2.html">내용2</a>]

In [18]:
dom.find(text=re.compile('내용'))

#정규식 표현도 사용 할 수 있다. 내용가지고 찝기도 하고, 어트리뷰트로 찝기도 하고.. 등등

'내용1'

find, find_all => 자식(recursive) 탐색

find_parent, find_parents => 부모

In [19]:
dom.a.find_parents()

[<li><a href="link1.html">내용1</a></li>,
 <ul>
 <li><a href="link1.html">내용1</a></li></ul>,
 <body>
 <ul>
 <li><a href="link1.html">내용1</a></li></ul></body>,
 <html>
 <head></head>
 <body>
 <ul>
 <li><a href="link1.html">내용1</a></li></ul></body></html>,
 
 <html>
 <head></head>
 <body>
 <ul>
 <li><a href="link1.html">내용1</a></li></ul></body></html>
 <li><a href="link2.html">내용2</a></li>
 <li><a href="link3.html">내용3</a></li>
 <li><a href="link4.html">내용4</a></li>
 
 
 
 ]

-----

In [20]:
from requests import request, HTTPError
from requests.compat import urljoin,urlparse,quote,unquote
from time import sleep

def download(url, params={},headers={},method="GET"):
    resp = request(url=url, params=params if method=="GET" else{},
                   data=params if method =="POST" else{},
                   headers=headers, method=method)
    
    try:
        resp.raise_for_status()
    except HTTPError as e:
        if 500 <= e.response.status_code < 600 and maximum >0:
            print('try again')
            sleep(1)
            resp = download(url,params,headers,method,maximum-1)
        else:
            print(e)
        
    return resp

In [21]:
url = 'https://pythonscraping.com/pages/page3.html'
resp = download(url)
dom = BeautifulSoup(resp.text, 'html.parser')

In [22]:
for _ in dom.body.div.find_all(recursive=False):
    print(_.name)

img
h1
div
table


In [23]:
url = 'https://pythonscraping.com/pages/page3.html'
resp = download(url)
dom = BeautifulSoup(resp.text, 'html5lib')

In [24]:
for _ in dom.body.div.find_all(recursive=False):
    print(_.name)

img
h1
div
table
p
div


상황에 맞춰서 써야한다. 파서를 써야할 상황도 있고, 립을 써야하는 상황도 있다.

--------

특정 값을 찾는 연습!

In [25]:
url = 'https://pythonscraping.com/pages/page3.html'
resp = download(url)
dom = BeautifulSoup(resp.text, 'html.parser')

In [26]:
base = dom.find(attrs={'id':'footer'})

In [27]:
base

<div id="footer">
© Totally Normal Gifts, Inc. <br/>
+234 (617) 863-0736
</div>

In [28]:
[_.name for _ in base.find_previous_siblings()]

['html']

In [29]:
base.find_previous_sibling().find().find_next_sibling()\
.find().find_all(recursive=False)[-1].find_all(recursive=False)[1]\
.find_all(recursive=False)[2].text.strip()

'$15.00'

In [30]:
dom.find('table').find_all(recursive=False)
#이렇게 하면 모든 행을 볼 수 있다

[<tr><th>
 Item Title
 </th><th>
 Description
 </th><th>
 Cost
 </th><th>
 Image
 </th></tr>,
 <tr class="gift" id="gift1"><td>
 Vegetable Basket
 </td><td>
 This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
 <span class="excitingNote">Now with super-colorful bell peppers!</span>
 </td><td>
 $15.00
 </td><td>
 <img src="../img/gifts/img1.jpg"/>
 </td></tr>,
 <tr class="gift" id="gift2"><td>
 Russian Nesting Dolls
 </td><td>
 Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
 </td><td>
 $10,000.52
 </td><td>
 <img src="../img/gifts/img2.jpg"/>
 </td></tr>,
 <tr class="gift" id="gift3"><td>
 Fish Painting
 </td><td>
 If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
 </td><td>
 $10,005.00
 </td><td>
 <img

In [31]:
for _ in dom.find('table').find_all(recursive=False):
    if _.find('td'):
        print(_.find_all('td')[2].text.strip())
        
#각 가격을 이렇게 출력할 수 있다

$15.00
$10,000.52
$10,005.00
$0.50
$1.50


In [32]:
for _ in dom.find('table').find_all(recursive=False):
    if _.find('td'):
        print(_.find_all('td')[-1].find('img').attrs['src'])
        print(urljoin(resp.request.url,
                     _.find_all('td')[-1].find('img').attrs['src']))
    else:
        print(_.find_all('th')[-1].text.strip())

Image
../img/gifts/img1.jpg
https://pythonscraping.com/img/gifts/img1.jpg
../img/gifts/img2.jpg
https://pythonscraping.com/img/gifts/img2.jpg
../img/gifts/img3.jpg
https://pythonscraping.com/img/gifts/img3.jpg
../img/gifts/img4.jpg
https://pythonscraping.com/img/gifts/img4.jpg
../img/gifts/img6.jpg
https://pythonscraping.com/img/gifts/img6.jpg


-------

7월 8일 목요일 아침 10시

http 가지고 리퀘스트 보내고 리스펀스 보내는거 어제 했다.
항상 바이트형태의 스트링으로 날라가기 때문에 변환이 필요하다! 유니코드로!
리퀘스트라는 패키지를 썼다 -
다운로드 함수도 만들었다
에러는 400번대 에러, 500번대 에러도 공부했었다
xml, json 타입의 마크업 문서들을 어떻게 파싱할것이냐 - 이제 뷰티풀숩으로 돔을 생성해서 한다!
돔을 파싱할때 어제 해본건 특정 노드의 구조를 통해 찾는 연습을 했었다~

최대한 많이 연습을 하다보면 자연스럽게 익혀진다.

------

In [52]:
from requests import Response , request

In [59]:
url='https://www.google.com/search'
params={
     'q' : '파이썬',
     'ei': 'f0jlYOetBJWUr7wPl8uE0Aw',
     'oq': '파 이 썬',
     'gs_lcp': 'Cgdnd3Mtd2l6EAMyAggmMgIIJjoHCAAQRxCwA0oECEEYAFDZ6gZYme0GYLv0BmgBcAJ4AIABtAGIAY8EkgEDMC4zmAEAoAEBqgEHZ3dzLXdpesgBCMABAQ',
     'sclient': 'gws-wiz',
     'ved': '0ahUKEwin6dOrqdDxAhUVyosBHZclAcoQ4dUDCA4',
     'uact': '5'    
}

params['q'] = '파이썬'

resp = download(url, params, header, 'GET')

NameError: name 'header' is not defined

In [ ]:
dom = BeautifulSoup(resp.text, 'lxml')

In [ ]:
dom.find(attrs={'id':'rso'})
[_.name for _ in result.find_all(recursive=False)]